### Conversational QnA chabot using RAG

In [38]:
## Importing library
import os
from dotenv import load_dotenv
load_dotenv()
from pprint import pprint
import bs4
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
 

In [26]:
## Calling LLM
groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025906990520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025906991090>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
## Importing Embedding from HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings
os.environ['HF_TOKEN']="Put your HuggingFace Token"
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [29]:
## Load, chunk and index the contents of the blog to create a retriever.

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Board_of_Control_for_Cricket_in_India")

docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Board_of_Control_for_Cricket_in_India', 'title': 'Board of Control for Cricket in India - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nBoard of Control for Cricket in India - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [31]:
## after importing docs we split them then convert them to vector and store them in vector database called chroma 
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002595B641030>, search_kwargs={})

In [39]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain=create_stuff_documents_chain(llm,prompt) # Stuff document chains help to ingest prompt inside the LLM
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

response=rag_chain.invoke({"input":"What is Anto doping unit"})
pprint(response['answer'])

('The Anti-doping unit is a unit headed by Dr. Abhijit Salvi that collects '
 'blood and urine samples from players, analyzes them in a lab to detect '
 'performance-enhancing drugs or steroids, and takes appropriate actions if '
 'violations are found.')


In [43]:
response=rag_chain.invoke({"input":"How do we achieve it"})
pprint(response['answer'])

"I don't know"


In [44]:
#### We can see our model not able to pick the historical context

#### Let's add the chat history

In [47]:
from langchain.chains import create_history_aware_retriever # This helps in remembering the history
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [48]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002595B641030>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='A

In [50]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002595B641030>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], ty

In [69]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Anto doping unit?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

pprint(response1['answer'])

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="How does it works?"
response2=rag_chain.invoke({"input":question2,"chat_history":chat_history})
pprint(response2['answer'])

chat_history.extend(
    [
        HumanMessage(content=question2),
        AIMessage(content=response2["answer"])
    ]
)

('The anti-doping unit is a department within the BCCI (Board of Control for '
 'Cricket in India) responsible for detecting and preventing the use of '
 'performance-enhancing drugs or steroids in cricket.')
('The anti-doping unit works by sending qualified doctors from NADA (National '
 'Anti-Doping Agency) to collect blood and urine samples from players, which '
 'are then analyzed in a lab to detect the presence of performance-enhancing '
 'drugs or steroids. If any violations are found, the BCCI takes appropriate '
 'actions.')


In [70]:
chat_history

[HumanMessage(content='What is Anto doping unit?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The anti-doping unit is a department within the BCCI (Board of Control for Cricket in India) responsible for detecting and preventing the use of performance-enhancing drugs or steroids in cricket.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How does it works?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The anti-doping unit works by sending qualified doctors from NADA (National Anti-Doping Agency) to collect blood and urine samples from players, which are then analyzed in a lab to detect the presence of performance-enhancing drugs or steroids. If any violations are found, the BCCI takes appropriate actions.', additional_kwargs={}, response_metadata={})]

#### Storing the chat history is sessions

In [71]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [74]:
pprint(conversational_rag_chain.invoke(
    {"input": "What is Virat Kohli sacking?"},
    config={
        "configurable": {"session_id": "xyz"}
    },  
)["answer"])

('According to the context, Virat Kohli was sacked from the ODI captaincy by '
 'the BCCI in 2021.')


In [77]:
pprint(conversational_rag_chain.invoke(
    {"input": "whay was virat sacked?"},
    config={
        "configurable": {"session_id": "xyz"}
    },  
)["answer"])

('According to the context, Virat Kohli was sacked because he was considering '
 'himself bigger than BCCI and wanted to teach a lesson to the president of '
 'BCCI, as revealed by Chetan Sharma, the then chief national selector, in a '
 'sting operation.')


In [81]:
get_session_history('xyz').messages

[HumanMessage(content='What is Conflict of interest?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the context, a Conflict of Interest is a situation where an individual has a personal or financial stake in a decision made by them, which could influence their judgment and lead to unfair or biased outcomes.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is Conflict of interest?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I don't know. The provided context only explains that the BCCI has an ethics officer to review complaints of conflict of interest, but it does not define what a conflict of interest is.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is Virat Kohli sacking?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the context, Virat Kohli was sacked from the ODI captaincy by the BCCI in 2021.', additional_kwargs={}, response_metadata={}),